In [ ]:
from influxdb_client import InfluxDBClient

import pandas as pd

from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
#Auth parameters
my_token = "yaKfFeAsha8tNAZxvYeZBMmq-khO8tz-6Ut_PARgohiWzeW2j8BB86ND33Qbq7hR8bylmOROPQAUr-7M103_Yw=="
my_org = "iot-org"
bucket = "iot-demo"
client = InfluxDBClient(url="http://192.168.1.100:9999", token=my_token, org=my_org, debug=False)

In [ ]:
days = '5d'
n_days = 5

queryTemperature = '''
from(bucket: "iot-demo")
  |> range(start: -%s, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "temperature")
  |> filter(fn: (r) => r["device"] == "RoomWeather")
  |> aggregateWindow(every: 10m, fn: median)
  ''' % days

queryHumidity = '''
from(bucket: "iot-demo")
  |> range(start: -%s, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "humidity")
  |> filter(fn: (r) => r["device"] == "RoomWeather")
  |> aggregateWindow(every: 10m, fn: median)
  ''' % days

queryPressure = '''
from(bucket: "iot-demo")
  |> range(start: -%s, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "pressure")
  |> filter(fn: (r) => r["device"] == "RoomWeather")
  |> aggregateWindow(every: 10m, fn: median)
  ''' % days

In [ ]:
#Query InfluxDB and return a Dataframe
dfTemp = client.query_api().query_data_frame(org=my_org, query=queryTemperature)
dfHum = client.query_api().query_data_frame(org=my_org, query=queryHumidity)
dfPress = client.query_api().query_data_frame(org=my_org, query=queryPressure)

In [ ]:
df = pd.DataFrame(dfTemp['_time']).rename(columns={'_time':'time'})
df['temperature'] = dfTemp['_value']
df['humidity'] = dfHum['_value']
df['pressure'] = dfPress['_value']
df = df.set_index('time')
df = df.asfreq(freq='600S')
df.head()

In [ ]:
points_per_day = int(len(df)/n_days) 

df['temperature'] = df['temperature'].fillna(method='backfill')
df['humidity'] = df['humidity'].fillna(method='backfill')
df['pressure'] = df['pressure'].fillna(method='backfill').fillna(method='ffill')

print(df['temperature'].isnull().sum())
print(df['humidity'].isnull().sum())
print(df['pressure'].isnull().sum())

In [ ]:
# divide in train e test
size = int(len(df)*0.8)
df_train, df_test = df.iloc[:size], df.iloc[size:]

In [ ]:
print(df_test.index[0])
print(df_test.index[len(df_test)-1])

In [ ]:
model_p = SARIMAX(df_train.pressure, order = (1,1,1))
results_p = model_p.fit()
print(results_p.summary())

In [ ]:
print(df_test.index[0])
print(df_test.index[len(df_test)-1])
start_date = df_test.index[0]
end_date = df_test.index[len(df_test)-1]

In [ ]:
df_pred_p = results_p.predict(start = start_date, end = end_date)

df_pred_p[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.pressure[start_date:end_date].plot(color = "blue")
plt.title("Predictions vs Actual", size = 24)
plt.show()

In [ ]:
print("ABSOLUTE ERROR "+str(mean_absolute_error(df_test.pressure[start_date:end_date], df_pred_p[start_date:end_date])))
print("MSE "+str(mean_squared_error(df_test.pressure[start_date:end_date], df_pred_p[start_date:end_date])))

In [ ]:
results_t.plot_diagnostics(figsize = (30,20))
plt.show()